# gensimによるトピックモデル入門

## ファイルを読み込む

題材は国会（衆議院）の会議録データ<br/>
テキストはあらかじめ形態素解析（単語分割）済み<br/>
今回はgensimの機能を使ってテキストを処理する

In [1]:
# ファイル一覧を作成する
import glob

txt_list = glob.glob('diet/189*.txt')

In [2]:
documents = []
all_texts = []

# ファイルごとに分けて読み込む
for f in txt_list:
    with open(f, encoding='utf-8') as fh:
        # 一気に全部読み込み（リストに1行ずつ入る）
        t = fh.readlines()
    
    # 行を全部つなげて，改行コードを抜いた上で，単語に分割して格納
    documents.append(" ".join(t).replace('\n', '').split())
    # さらに全テキストにも加えておく
    all_texts += documents[-1]

In [3]:
# ファイル（文書）の数と総単語数
doc_size = len(documents)
text_size = len(all_texts)

print(doc_size)
print(text_size)

44
2602496


In [2]:
# ファイル名が番号でわかりにくいので，会議コードと会議名の対応（辞書）を作っておく
code = {}

# このファイルには4桁の会議コードと対応する会議名が格納されている
with open('diet/code.txt', encoding='utf-8') as fh:
    for s in fh:
        t = s.rstrip('\n').split(' ')
        if len(t) < 2:
            continue
        code[t[0]] = t[1]
print(code)

{'0001': '本会議', '0002': '内閣', '0094': '総務', '0004': '法務', '0005': '外務', '0095': '財務金融', '0096': '文部科学', '0097': '厚生労働', '0009': '農林水産', '0098': '経済産業', '0099': '国土交通', '0017': '環境', '0015': '安全保障', '0087': '国家基本政策', '0018': '予算', '0058': '決算', '0041': '決算一分科', '0042': '決算二分科', '0043': '決算三分科', '0044': '決算四分科', '0020': '議院運営', '0021': '懲罰', '0022': '災害対策', '0071': '倫理選挙', '0025': '沖縄北方', '0073': '青少年', '0202': '海賊テロ', '0142': '拉致', '0197': '消費者', '0003': '地方行政', '0006': '大蔵', '0007': '文教', '0008': '厚生', '0010': '商工', '0011': '運輸', '0012': '逓信', '0013': '労働', '0014': '建設', '0016': '科学技術', '0233': '科学技術', '0133': 'イラク', '0151': '憲法', '0178': '教育再生', '0023': '石炭対策', '0026': '国会移転', '0104': 'テロ', '0122': '法務厚労連合', '0123': '特殊法人改革', '0130': '個人情報保護', '0132': 'イラク・テロ', '0024': '消費者', '0112': '武力事態', '0134': '武力事態等', '0146': '郵政民営化', '0028': '行政改革', '0158': '教育基本法', '0072': '日米防衛協力', '0089': '憲法調査会', '0088': '国家基本合同', '0143': '年金社会保障', '0082': '予算合同審査', '0242': '震災復興', '0030': '予算公聴会', '0031':

## テキスト全体の概要

まず辞書（Python標準のdictクラスではなく，gensimにおける単語とIDの対応を取るもの）を作る

In [5]:
from gensim import corpora

# 文書データから辞書を作成
dictionary = corpora.Dictionary(documents)

# 保存もできる（単語ID・単語・文書頻度すなわちDFの3つ組が保存される）
dictionary.save_as_text('diet/dict.txt')

In [6]:
# 全テキストをBoWにすると，各単語の出現数を数えたことになる
bow = dictionary.doc2bow(all_texts)

# BoWに含まれる単語の種類数（語彙サイズ）は
vocab_size = len(bow)
print(vocab_size)

# 先頭の10単語を眺めてみる（単語IDと頻度）
# 自然言語処理の世界では，単語をIDすなわち番号で表す習慣（というより必要性）が昔からあった
bow[0:10]

21103


[(0, 181946),
 (1, 60013),
 (2, 159),
 (3, 88),
 (4, 23),
 (5, 2),
 (6, 93),
 (7, 4),
 (8, 167),
 (9, 34)]

In [7]:
# 単語がID化されているので，単語そのものに戻しておく
all_texts_wf = [ (dictionary.get(w[0]), w[1]) for w in dictionary.doc2bow(all_texts) ]

# すると先ほどの単語10個は
all_texts_wf[0:10]

[('、', 181946),
 ('。', 60013),
 ('〇', 159),
 ('ああ', 88),
 ('あい', 23),
 ('あえい', 2),
 ('あえて', 93),
 ('あおる', 4),
 ('あくまで', 167),
 ('あけ', 34)]

In [8]:
# 頻度を降順に並べ替えて眺めてみる
all_texts_wf_sorted = sorted(all_texts_wf, key=lambda wf: wf[1], reverse=True)
for i in range(20):
    print(all_texts_wf_sorted[i])

('、', 181946)
('の', 114077)
('て', 111441)
('に', 95184)
('と', 88546)
('を', 66160)
('で', 63486)
('は', 62802)
('。', 60013)
('が', 55432)
('ます', 50638)
('いう', 45369)
('し', 38649)
('も', 37256)
('た', 36754)
('こと', 27443)
('な', 22621)
('か', 21923)
('まし', 21051)
('です', 19290)


## 文書の表現（ベクトル空間モデル）

単語頻度から各文書を表すベクトルを作る

In [9]:
# 今度は文書ごとにBoWにする
doc_bow = [ dictionary.doc2bow(d) for d in documents ]

# 保存もできる
corpora.MmCorpus.serialize('diet/diet.mm', doc_bow)

In [10]:
# ある1つの文書の単語頻度を今度は昇順に並べ替えて，眺めてみる
# たとえば最初（0番目）の文書
doc_id = 0

wv = [ (dictionary.get(w[0]), w[1]) for w in sorted(doc_bow[doc_id], key=lambda wf: wf[1]) ]

# この文書の単語の種類数
# 文書に出現した単語しか含まれない（したがって全体の語彙サイズより小さい）ことに注意
print(len(wv))

# スペースの節約のため，つなげてから表示
wv_seq = ""
for i in range(500):
    # 単語の後にコロン（：）で区切って頻度を付けて表示
    wv_seq += " " + wv[i][0] + ":" + str(wv[i][1])
print(wv_seq)

4458
 ああ:1 あえい:1 あおる:1 あけ:1 あける:1 あげる:1 あたかも:1 あっさり:1 あら:1 あらあら:1 あらかじめ:1 あらわさ:1 あらわし:1 あらわす:1 あらわれ:1 ありあり:1 ありさま:1 あんまり:1 いかん:1 いきなり:1 いこ:1 いち早く:1 いっそ:1 いっぱい:1 いとま:1 いみじく:1 いよう:1 いらっしゃら:1 うかがい:1 うそ:1 うたわ:1 うっすら:1 おか:1 おく:1 おこがましい:1 おさまっ:1 おさめる:1 おちょくら:1 おっしゃら:1 おととし:1 おはよう:1 おまけ:1 おもしろい:1 およそ:1 かい:1 かかり:1 かかわり:1 かさ:1 かす:1 かなわ:1 かね:1 かねて:1 かねる:1 からだ:1 かりる:1 かんかん:1 がくがく:1 がたい:1 がん:1 きずな:1 きっと:1 きつい:1 きる:1 くださ:1 くださら:1 くださる:1 くだされ:1 くだん:1 くらまし:1 くれれ:1 ぐっすり:1 ぐったり:1 ぐっと:1 こころ:1 こそっ:1 こだわる:1 こちら側:1 こっち:1 こなし:1 ごく:1 さかのぼっ:1 さすれ:1 さておき:1 さらさ:1 さらさら:1 さらす:1 さらっと:1 さりとて:1 しね:1 しまわ:1 しょう:1 すぐれ:1 すごい:1 すさまじい:1 すばらしい:1 ずる:1 ずれ:1 ずれ込む:1 せっかく:1 せんだって:1 ぜいたく:1 そ:1 そご:1 そちら:1 そん:1 ぞ:1 ぞっと:1 たかっ:1 たくし:1 たけれ:1 たじたじ:1 たたき台:1 たださ:1 たって:1 たつ:1 たどり着け:1 たびたび:1 たまたま:1 たまっ:1 だり:1 ちなみ:1 ちゃい:1 ちゃえ:1 ちょうだい:1 っけ:1 ついに:1 つか:1 つかん:1 つく:1 つくり上げ:1 つける:1 つとめる:1 つなぎ:1 つなぐ:1 つなげ:1 つぶやか:1 できれ:1 でき上がっ:1 とう:1 とうとび:1 とうとぶ:1 とどまっ:1 とどまら:1 とどまる:1 とまっ:1 とら:1 どぶ:1 なかんずく:1 なき:1 なくなる:1 なさっ:1 なじむ:1 なでしこ:1 なるべく:1 なれる:1 にぎやかし:1 ぬん:1

In [11]:
# データフレームに変換する場合
import numpy as np
from pandas import DataFrame 

# 空（0）の行列を用意して，各文書で単語が出現していた場合だけその頻度を代入
word_vect = np.zeros(doc_size * vocab_size, dtype=np.int32).reshape(doc_size, vocab_size)
for i in range(doc_size):
    for w, f in doc_bow[i]:
        word_vect[i][w] = f

# データフレームに変換して列名に単語を設定
word_df = DataFrame(word_vect)
word_df.columns=[ dictionary.get(i) for i in range(vocab_size)]
word_df

,、,。,〇,ああ,あい,あえい,あえて,あおる,あくまで,あけ,...,遊具,都議,長森,開披,駆け引き,ＡＩＩ,ＧＩＨ,ＩＢ,ＮＡＣＣＳ,ＴＰＡ
0,5978,1937,8,1,2,1,9,1,4,1,...,0,0,0,0,0,0,0,0,0,0
1,6305,2122,5,3,0,0,12,0,8,0,...,0,0,0,0,0,0,0,0,0,0
2,9504,3096,7,13,0,0,3,0,15,0,...,0,0,0,0,0,0,0,0,0,0
3,4322,1501,2,2,0,0,1,0,8,0,...,0,0,0,0,0,0,0,0,0,0
4,6555,2289,3,1,0,0,4,0,5,1,...,0,0,0,0,0,0,0,0,0,0
5,4636,1563,21,0,0,1,1,0,5,0,...,0,0,0,0,0,0,0,0,0,0
6,3282,1052,4,3,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8622,2643,5,3,0,0,7,0,13,1,...,0,0,0,0,0,0,0,0,0,0
8,8984,2825,5,5,1,0,7,0,4,3,...,0,0,0,0,0,0,0,0,0,0
9,9662,2764,1,5,0,0,1,1,7,6,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# この単語頻度ベクトルがそれぞれの文書を表しているので，これにより分類などを行うことが（いちおう）できる

# 読点「、」と句点「。」がどの文書でも著しく頻度が大きいので，あらかじめ抜いておく
del word_df['、']
del word_df['。']

word_vect = np.array(word_df)

In [13]:
# コサイン類似度を定義して
def cos_similarity(v1, v2):
    return np.dot(v1, v2) / np.linalg.norm(v1) / np.linalg.norm(v2)

# 文書間の類似度を計算
wv_sims = [ [ cos_similarity(i, j) for j in word_vect ] for i in word_vect  ]

In [14]:
# たとえば最初の文書と各文書との類似度を見ると，どれもあまり変わらない…
# （これは，句読点は外したが，どの文書にも出てくる機能語などはそのままにしたため）
wv_sims[0]

[1.0,
 0.9879127147459573,
 0.9938848314772224,
 0.9938526094339409,
 0.9919735405624218,
 0.9868514298950202,
 0.9876588881243579,
 0.9938511354033398,
 0.9943947823202511,
 0.9927675476830887,
 0.9937318901321192,
 0.9912140929833283,
 0.9928043189640616,
 0.9924664032764902,
 0.9877912661691689,
 0.9925211290201399,
 0.9873560989331148,
 0.9120771354550427,
 0.9863007988899939,
 0.985275829041148,
 0.9917171343947517,
 0.9731912558822993,
 0.9812410019010976,
 0.9886792692570427,
 0.9916002404743375,
 0.9855451884829975,
 0.9847793438015741,
 0.9781227809994809,
 0.9877538810793393,
 0.9926934610859643,
 0.9937444679131278,
 0.9820709324375003,
 0.9586305807981093,
 0.9882077703261116,
 0.9915564770170395,
 0.9856288716553062,
 0.9558179095825727,
 0.9840338134796613,
 0.9887093307668091,
 0.9878633206273574,
 0.9900577788679544,
 0.9825791232789958,
 0.9894353378189868,
 0.9898635863448069]

## TF-IDFの計算

単純な単語頻度のままではうまく表せないので，単語を重み付けする

In [15]:
from gensim import models

# 読み込んだ文書データからIDFを計算
tfidf = models.TfidfModel(doc_bow)

# これをBoWのデータ（つまりTF）に適用するとTF-IDFが計算できる
# 同じ文書集合で計算してみる（もちろん別の文書でもできる）
tfidf_vect = tfidf[doc_bow]

# たとえば最初の文書について，先ほどのBoWとTF-IDFとで単語の種類数を見てみると
print(len(doc_bow[0]))
print(len(tfidf_vect[0]))

# 大きさが異なる（TF-IDFの種類の方が少ない）ことに注意
# これは，TF-IDFの計算時に，全文書に出てくる単語が（TF-IDFが0になるので）省かれるため

4458
4182


In [16]:
# ある1つの文書の単語をいくつか抜き出してTF-IDF値を見てみる
# （値が正規化されていることに注意）
# たとえば最初（0番目）の文書
doc_id = 0

for i in range(0, 3000, 50):
    print("%-8s\t%.6f" % (dictionary.get(tfidf_vect[doc_id][i][0]), tfidf_vect[doc_id][i][1]))

〇       	0.006885
いただこう   	0.004674
かかっ     	0.002812
ぐったり    	0.009300
しよう     	0.000904
ただいま    	0.001215
つなぐ     	0.003641
なくそう    	0.009793
ひずみ     	0.009300
みなし     	0.004897
よくよく    	0.009579
インターネット 	0.010801
サイクル    	0.009348
タンク     	0.027106
バンク     	0.007934
ベスト     	0.004393
ライフ     	0.002645
一本      	0.003407
下層      	0.009300
中日      	0.009300
事柄      	0.002197
他       	0.001640
伸ばす     	0.007282
価格      	0.008237
債権      	0.002337
入閣      	0.007596
内部      	0.002049
分権      	0.035146
副       	0.002811
務める     	0.002337
危惧      	0.001025
取ら      	0.025137
司令      	0.003407
否定      	0.001565
図ら      	0.001200
培い      	0.006600
変更      	0.002582
大量      	0.002337
孫請      	0.009300
寄り添っ    	0.002197
展開      	0.002250
席       	0.004972
廃案      	0.004190
当たり前    	0.003074
徴収      	0.018697
悪       	0.007971
所属      	0.000861
押し切ら    	0.009300
掘り下げ    	0.006600
放出      	0.004190
新年      	0.011786
普通      	0.002466
期限      	0.001389
柔軟      	0.005813
機械      	0.002586
残高      	0

In [17]:
# TF-IDFによる文書のベクトルが定義できたので，類似度を計算してみる
from gensim import similarities

index = similarities.MatrixSimilarity(tfidf_vect)

# 同じ文書集合に対する類似度（他の文書を用意して計算することも，もちろんできる）
sims = index[tfidf_vect]

In [18]:
# 最初の文書に関する類似度
print(sims[0])

# 単純な単語頻度ベクトルよりはメリハリがついた

[1.         0.08874273 0.15852731 0.21149567 0.15403493 0.0956249
 0.15364742 0.15858473 0.13060652 0.08450319 0.12751704 0.06570865
 0.12926412 0.06362168 0.14180663 0.11323403 0.08742797 0.08289602
 0.13741435 0.11469714 0.17259312 0.030237   0.03232662 0.16762033
 0.11819246 0.0561175  0.03748471 0.14501339 0.01998881 0.15744826
 0.17763776 0.03174082 0.04217741 0.20143926 0.09587648 0.05000528
 0.04859471 0.09551789 0.05588775 0.07707472 0.13061307 0.09406942
 0.06766438 0.0577356 ]


In [19]:
# わかりにくいので，文書ごとに類似度を降順に並び替え
sims_sorted = [ sorted(enumerate(s), key=lambda item: -item[1]) for s in sims ]

# 会議のIDと名前も入れておくことにする
doc_sims = [ [ (txt_list[t[0]], code[txt_list[t[0]][-11:-7]], t[1]) for t in s ] for s in sims_sorted ]

In [20]:
# 最初の文書に対する類似度の上位10件
print(txt_list[0], code[txt_list[0][-11:-7]])
for i in range(10):
    print(doc_sims[0][i])

diet\18903020018012.txt 予算
('diet\\18903020018012.txt', '予算', 1.0)
('diet\\18903050094003.txt', '総務', 0.21149567)
('diet\\18903240094009.txt', '総務', 0.20143926)
('diet\\18903200099002.txt', '国土交通', 0.17763776)
('diet\\18903130095005.txt', '財務金融', 0.17259312)
('diet\\18903190025003.txt', '沖縄北方', 0.16762033)
('diet\\18903100032001.txt', '予算二分科', 0.15858473)
('diet\\18903050018014.txt', '予算', 0.15852731)
('diet\\18903200098003.txt', '経済産業', 0.15744826)
('diet\\18903060018015.txt', '予算', 0.15403493)


In [21]:
# 別の文書
print(txt_list[22], code[txt_list[22][-11:-7]])
for i in range(10):
    print(doc_sims[22][i])

diet\18903180098002.txt 経済産業
('diet\\18903180098002.txt', '経済産業', 1.0000001)
('diet\\18903100037001.txt', '予算七分科', 0.17665246)
('diet\\18903200098003.txt', '経済産業', 0.1445419)
('diet\\18903270098005.txt', '経済産業', 0.11122918)
('diet\\18903200022003.txt', '災害対策', 0.07883493)
('diet\\18903270017003.txt', '環境', 0.070813216)
('diet\\18903240017002.txt', '環境', 0.05897405)
('diet\\18903260242003.txt', '震災復興', 0.053829536)
('diet\\18903060018015.txt', '予算', 0.052814312)
('diet\\18903100034001.txt', '予算四分科', 0.048320986)


## 次元の削減

ベクトルの次元が多すぎるので，頻度をもとに削減する

In [22]:
# 最初の辞書から，文書頻度が一定以下・一定以上のものを除去
dictionary.filter_extremes()

# 新しい辞書のサイズ
print(len(list(dictionary.values())))

# 保存もしておく
dictionary.save_as_text('diet/dict2.txt')

5092


In [23]:
# ベクトルを作り直す
word_vect2 = [ dictionary.doc2bow(d) for d in documents ]

# TF-IDFも計算し直す
tfidf2 = models.TfidfModel(word_vect2)
tfidf_vect2 = tfidf[word_vect2]

# 類似度も計算し直す
index2 = similarities.MatrixSimilarity(tfidf_vect2)
sims2 = index2[tfidf_vect2]

sims2_sorted = [ sorted(enumerate(s), key=lambda item: -item[1]) for s in sims2 ]
doc_sims2 = [ [ (txt_list[t[0]], code[txt_list[t[0]][-11:-7]], t[1]) for t in s ] for s in sims2_sorted ]

In [24]:
# さきほどの文書は
print(txt_list[22], code[txt_list[22][-11:-7]])
for i in range(10):
    print(doc_sims2[22][i])

# 頻度の小さな多数の単語を削除して次元を大きく削減し，かつ多くの文書に現れる単語を削除して
# 類似度計算から除外することで，類似度がもとのTF-IDFよりもさらにはっきりした

diet\18903180098002.txt 経済産業
('diet\\18903180098002.txt', '経済産業', 0.99999994)
('diet\\18903100037001.txt', '予算七分科', 0.2364548)
('diet\\18903270017003.txt', '環境', 0.20474873)
('diet\\18903200098003.txt', '経済産業', 0.17765844)
('diet\\18903270098005.txt', '経済産業', 0.13743325)
('diet\\18903240017002.txt', '環境', 0.13277115)
('diet\\18903200022003.txt', '災害対策', 0.10910436)
('diet\\18903100036001.txt', '予算六分科', 0.09667667)
('diet\\18903030018013.txt', '予算', 0.087910324)
('diet\\18903020018012.txt', '予算', 0.07864376)


## LDAモデル

In [25]:
# 次元を削減したデータでモデルを学習（トピック数はとりあえず10）
topic_num = 10
lda = models.LdaModel(word_vect2, num_topics=topic_num, passes=10)

In [26]:
# 先ほどと全く同じ要領でベクトルや類似度を計算
lda_vect = lda[word_vect2]
index3 = similarities.MatrixSimilarity(lda_vect)
sims3 = index3[lda_vect]
sims3_sorted = [ sorted(enumerate(s), key=lambda item: -item[1]) for s in sims3 ]
doc_sims3 = [ [ (txt_list[t[0]], code[txt_list[t[0]][-11:-7]], t[1]) for t in s ] for s in sims3_sorted ]

In [27]:
# 今度のベクトルは各トピックとの当てはまりの程度（確率）になるので，次元数はトピック数になる
# （ただし確率がほぼ0のトピックは省かれているので，実際には10個は表示されない）

# 44文書全てについて表示すると
for i, v in enumerate(lda_vect):
    print(i, code[txt_list[i][-11:-7]], v)

0 予算 [(1, 0.80669945), (4, 0.013466163), (5, 0.093448095), (6, 0.024123326), (8, 0.054711457)]
1 予算 [(7, 0.99983394)]
2 予算 [(0, 0.050122164), (4, 0.010271584), (7, 0.9238661), (8, 0.010513702)]
3 総務 [(5, 0.999714)]
4 予算 [(1, 0.012987485), (2, 0.04877328), (4, 0.027306737), (5, 0.055933636), (7, 0.7290693), (8, 0.095101796), (9, 0.027318703)]
5 予算公聴会 [(0, 0.99976534)]
6 総務 [(2, 0.8839416), (5, 0.11575463)]
7 予算二分科 [(2, 0.077638045), (4, 0.6027981), (5, 0.1568873), (7, 0.026118837), (8, 0.10701047), (9, 0.02931522)]
8 予算三分科 [(6, 0.99988693)]
9 予算四分科 [(4, 0.99989283)]
10 予算五分科 [(0, 0.99987453)]
11 予算六分科 [(8, 0.9998844)]
12 予算七分科 [(9, 0.9998819)]
13 財務金融 [(2, 0.9997394)]
14 財務金融 [(2, 0.99956787)]
15 予算 [(3, 0.99982923)]
16 総務 [(2, 0.9995382)]
17 本会議 [(1, 0.8671059), (2, 0.11397626), (3, 0.018388221)]
18 予算 [(1, 0.9190031), (3, 0.04389485), (4, 0.03682318)]
19 総務 [(1, 0.14236064), (2, 0.40438902), (5, 0.4300079), (9, 0.022828246)]
20 財務金融 [(1, 0.9455599), (2, 0.054165713)]
21 農林水産 [(0, 0.03

In [28]:
# さきほどの文書のLDAによる類似度は
print(txt_list[22], code[txt_list[22][-11:-7]])
for i in range(10):
    print(doc_sims3[22][i])

# さらに類似度がはっきりした

diet\18903180098002.txt 経済産業
('diet\\18903100037001.txt', '予算七分科', 1.0)
('diet\\18903180098002.txt', '経済産業', 1.0)
('diet\\18903200022003.txt', '災害対策', 1.0)
('diet\\18903240017002.txt', '環境', 1.0)
('diet\\18903260242003.txt', '震災復興', 1.0)
('diet\\18903270098005.txt', '経済産業', 1.0)
('diet\\18903200099002.txt', '国土交通', 0.99168843)
('diet\\18903200098003.txt', '経済産業', 0.48093408)
('diet\\18903270017003.txt', '環境', 0.10530194)
('diet\\18903100032001.txt', '予算二分科', 0.045944896)


## クラスタリング

このように判別能力の向上したベクトルで文書を表すことができたので，さまざまな機械学習手法が適用できる<br/>
ここではクラスタリングにより，トピックの類似した文書のクラスターができるか試してみる

In [29]:
# K平均法
from sklearn.cluster import KMeans

# 普通にベクトルを取り出すと値の小さいものが省かれてしまうので，全部取り出す
lda_full_vect = [ [ t[1] for t in lda.get_document_topics(v, minimum_probability=0) ] for v in word_vect2 ]

# Numpyのオブジェクトに変換して，数と次元も確認
lfv = np.array(lda_full_vect)
print(lfv.shape)

# クラスタリングを実行（クラスター数はひとまず10個）
lda_cluster = KMeans(n_clusters=10)
lda_cluster.fit(lfv)

(44, 10)


KMeans(n_clusters=10)

In [30]:
# 結果に会議の情報を付けて表示
sorted([ (txt_list[i], code[txt_list[i][-11:-7]], l) for i, l in enumerate(lda_cluster.labels_) ],
       key=lambda item: item[2])

[('diet\\18903090030001.txt', '予算公聴会', 0),
 ('diet\\18903100035001.txt', '予算五分科', 0),
 ('diet\\18903050094003.txt', '総務', 1),
 ('diet\\18903130094006.txt', '総務', 1),
 ('diet\\18903200094008.txt', '総務', 1),
 ('diet\\18903240094009.txt', '総務', 1),
 ('diet\\18903100032001.txt', '予算二分科', 2),
 ('diet\\18903100034001.txt', '予算四分科', 2),
 ('diet\\18903200025004.txt', '沖縄北方', 2),
 ('diet\\18903250002002.txt', '内閣', 2),
 ('diet\\18903250096002.txt', '文部科学', 2),
 ('diet\\18903270096003.txt', '文部科学', 2),
 ('diet\\18903100036001.txt', '予算六分科', 3),
 ('diet\\18903180009002.txt', '農林水産', 3),
 ('diet\\18903270287003.txt', '地方創生', 3),
 ('diet\\18903300018018.txt', '予算', 3),
 ('diet\\18903020018012.txt', '予算', 4),
 ('diet\\18903130001010.txt', '本会議', 4),
 ('diet\\18903130018017.txt', '予算', 4),
 ('diet\\18903130095005.txt', '財務金融', 4),
 ('diet\\18903200098003.txt', '経済産業', 4),
 ('diet\\18903270017003.txt', '環境', 4),
 ('diet\\18903030018013.txt', '予算', 5),
 ('diet\\18903050018014.txt', '予算', 5),
 ('diet\\1

## トピックワード

In [31]:
# LDAで学習された各トピックを特徴付ける単語（それぞれ重み，すなわち確率を持っている）
# クラスタリングにおけるクラスタの番号とは全く関係がないので注意
topic_words = []
for i in range(topic_num):
    topic_words.append(" ".join([ "%s@%.3f" % (dictionary.get(t[0]), t[1]) for t in lda.get_topic_terms(i) ] ))
    print("%d: %s" % (i, topic_words[-1]))

0: 症@0.015 医師@0.010 認知@0.009 報酬@0.009 診療@0.008 就労@0.006 監査@0.006 患者@0.005 社員@0.004 モデル@0.004
1: バイオ@0.006 増税@0.006 減税@0.005 医薬@0.005 薬物@0.005 東電@0.005 下請@0.004 廃棄@0.004 汚染@0.004 歳出@0.004
2: 減税@0.011 債@0.011 控除@0.009 投票@0.009 ＮＩＳＡ@0.009 納税@0.007 歳出@0.006 税額@0.006 転嫁@0.006 国税@0.005
3: 防衛@0.019 遺族@0.012 統制@0.011 寄附@0.010 博@0.010 友@0.010 献金@0.009 補佐@0.009 コントロール@0.008 ＧＰＩＦ@0.007
4: 文部@0.009 友@0.008 博@0.008 寄附@0.008 生徒@0.008 会費@0.007 教員@0.007 文科@0.006 消防@0.006 高校@0.006
5: 放送@0.065 ＮＨＫ@0.050 籾井@0.020 監査@0.017 受信@0.012 番組@0.012 弁護@0.007 視聴@0.006 秘書@0.006 不偏@0.004
6: 法務@0.016 裁判@0.012 外務@0.010 司法@0.009 少年@0.008 犯罪@0.007 人権@0.007 面会@0.006 検察@0.006 北朝鮮@0.005
7: 防衛@0.011 外務@0.009 基地@0.006 憲法@0.006 隊@0.006 武力@0.006 米軍@0.006 献金@0.006 領土@0.006 寄附@0.006
8: 農地@0.014 農協@0.013 農家@0.011 輸出@0.010 米@0.010 組合@0.006 特区@0.005 森林@0.005 担い手@0.005 販売@0.004
9: 発電@0.017 防災@0.012 避難@0.008 汚染@0.007 風力@0.007 地震@0.007 稼働@0.006 除染@0.006 電気@0.005 燃料@0.004
